In [101]:
import os
import redis
import hashlib
from metaflow import FlowSpec, step, S3
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import pickle

# Ejemplo de predicción de modelo usando Redis

Este notebook muestra cómo obtener predicciones de un modelo que produce predicciones en lotes. Las predicciones se cargaron en Redis. 

En este notebook, recuperamos las predicciones del modelo almacenadas en Redis. Los datos deben ser leídos, convertidos a cadenas y luego hasheados. Con este enfoque, podemos verificar si los datos existen en Redis y recuperar la predicción correspondiente. En caso de que los datos no existan, asignamos un valor de cero. Esta estrategia simula cómo podría comportarse un servicio en producción ante casos no contemplados.

La ventaja de utilizar Redis en este contexto radica en su capacidad para almacenar datos de forma eficiente en memoria, lo que permite un acceso rápido a las predicciones previamente calculadas. Además, Redis ofrece funcionalidades de almacenamiento en caché y persistencia de datos, lo que garantiza la disponibilidad y la integridad de las predicciones incluso en entornos de producción de alta demanda.

In [102]:
# Conectamos al servidor redis (asegúrate de que el docker compose esté corriendo)
r = redis.Redis(host='localhost', port=6379, decode_responses=True)

# Configuración de las credenciales de acceso a AWS S3 (minio)
os.environ['AWS_ACCESS_KEY_ID'] = "minio"
os.environ['AWS_SECRET_ACCESS_KEY'] = "minio123"
os.environ['AWS_ENDPOINT_URL_S3'] = "http://localhost:9000"

In [103]:
# Obtención de datos para prueba
df = pd.read_csv("./breast_cancer.csv", header=None)

# Sampleamos 100 valores al azar
df_temp = df.sample(50)

test_values = df_temp.values.tolist()

s3 = S3(s3root="s3://amqtp/")
scaler_obj = s3.get("scaler.pkl")
with open(scaler_obj.path, 'rb') as f:
    scaler = pickle.load(f)

# Aplicamos el scaler a los datos
scaled_values = scaler.transform(df_temp)


# Conversión de valores a cadenas y hash
# Esto debería implementarse en el pipeline. Dado que los números de punto flotante pueden 
# presentar problemas debido a pequeñas variaciones, se podría considerar redondearlos.
string_values = [' '.join(map(str, sublist)) for sublist in scaled_values]
hashed_values = [hashlib.sha256(substring.encode()).hexdigest() for substring in string_values]

# Inicializamos un diccionario para almacenar las salidas del modelo
model_outputs = {}

# Obtenemos las predicciones almacenadas en Redis
for hash_key in hashed_values:
    model_outputs[hash_key] = r.hgetall(f"predictions:{hash_key}")

/opt/anaconda3/lib/python3.11/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Veamos la salida del modelo para diferentes entradas:

In [104]:
print("Salidas de los modelos para las primeras 5 entradas:")
for index, test_value in enumerate(test_values):
    hash_key = hashed_values[index]
    tree_prediction = model_outputs[hash_key].get('tree', 'No disponible')
    svc_prediction = model_outputs[hash_key].get('svc', 'No disponible')
    knn_prediction = model_outputs[hash_key].get('knn', 'No disponible')
    reglog_prediction = model_outputs[hash_key].get('reglog', 'No disponible')
    
    print(f"\nPara la entrada: {test_value}")
    print(f"El modelo tree predice: {tree_prediction}")
    print(f"El modelo svc predice: {svc_prediction}")
    print(f"El modelo knn predice: {knn_prediction}")
    print(f"El modelo reglog predice: {reglog_prediction}")

print("\nSe han mostrado las predicciones para las primeras 5 entradas.")

Salidas de los modelos para las primeras 5 entradas:

Para la entrada: ['22.27', '19.67', '152.8', '1509.0', '0.1326', '0.2768', '0.4264', '0.1823', '0.2556', '0.07039', '1.215', '1.545', '10.05', '170.0', '0.006515', '0.08668', '0.104', '0.0248', '0.03112', '0.005037', '28.4', '28.01', '206.8', '2360.0', '0.1701', '0.6997', '0.9608', '0.291', '0.4055', '0.09789']
El modelo tree predice: Maligno
El modelo svc predice: Maligno
El modelo knn predice: Maligno
El modelo reglog predice: Maligno

Para la entrada: ['17.99', '20.66', '117.8', '991.7', '0.1036', '0.1304', '0.1201', '0.08824', '0.1992', '0.06069', '0.4537', '0.8733', '3.061', '49.81', '0.007231', '0.02772', '0.02509', '0.0148', '0.01414', '0.003336', '21.08', '25.41', '138.1', '1349.0', '0.1482', '0.3735', '0.3301', '0.1974', '0.306', '0.08503']
El modelo tree predice: Maligno
El modelo svc predice: Maligno
El modelo knn predice: Maligno
El modelo reglog predice: Maligno

Para la entrada: ['10.49', '19.29', '67.41', '336.1', '0.